In [1]:
import psycopg2
import pandas as pd

In [2]:
def create_database():
    # connect to default database
    # conn = psycopg.2connect("host=127.0.0.1 dbname=studentdb user=student password=student")
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=9310@Zkapffpdhs")
    conn.set_session(autocommit=True)
    cur=conn.cursor()
    
    # create sparkify database with UTF8 encoding
    cur.execute("DROP DATABASE IF EXISTS accounts")
    cur.execute("CREATE DATABASE accounts")
    
    # close connection to default database
    conn.close()
    
    # connect to sparkify database
    conn = psycopg2.connect("host=127.0.0.1 dbname=accounts user=postgres password=9310@Zkapffpdhs")
    cur = conn.cursor()
    
    return cur, conn

In [3]:
def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [4]:
def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [5]:
AccountsCountry = pd.read_csv("Datasets/Wealth-AccountsCountry.csv")

In [6]:
AccountsCountry.head()

,Country Code,Short Name,Table Name,Long Name,2-alpha code,Currency Unit,Special Notes,Region,Income Group,WB-2 code,...,PPP survey years,Balance of Payments Manual in use,External debt Reporting status,System of trade,Government Accounting concept,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Latest water withdrawal data
0,ALB,Albania,Albania,Republic of Albania,AL,Albanian lek,NaN,Europe & Central Asia,Upper middle income,AL,...,Rolling,BPM6,Actual,Special trade system,Consolidated central government,Yes,2012,2013.0,2018.0,2006.0
1,ARG,Argentina,Argentina,Argentine Republic,AR,Argentine peso,NaN,Latin America & Caribbean,Upper middle income,AR,...,NaN,BPM6,Actual,Special trade system,Consolidated central government,Yes,2008,2002.0,2018.0,2011.0
2,ARM,Armenia,Armenia,Republic of Armenia,AM,Armenian dram,NaN,Europe & Central Asia,Upper middle income,AM,...,2011,BPM6,Actual,General trade system,Consolidated central government,Yes,2014,NaN,2018.0,2012.0
3,AUS,Australia,Australia,Commonwealth of Australia,AU,Australian dollar,Fiscal year end: June 30; reporting period for...,East Asia & Pacific,High income,AU,...,2011,BPM6,NaN,General trade system,Consolidated central government,Yes,2015-2016,2013.0,2018.0,2013.0
4,AUT,Austria,Austria,Republic of Austria,AT,Euro,A simple multiplier is used to convert the nat...,Europe & Central Asia,High income,AT,...,Rolling,BPM6,NaN,Special trade system,Consolidated central government,Yes,2010,2014.0,2018.0,2010.0


In [7]:
AccountsCountry_clean = AccountsCountry[['Country Code', 'Short Name', 'Table Name', 'Long Name','Currency Unit']]

In [8]:
AccountsCountry_clean.head()

,Country Code,Short Name,Table Name,Long Name,Currency Unit
0,ALB,Albania,Albania,Republic of Albania,Albanian lek
1,ARG,Argentina,Argentina,Argentine Republic,Argentine peso
2,ARM,Armenia,Armenia,Republic of Armenia,Armenian dram
3,AUS,Australia,Australia,Commonwealth of Australia,Australian dollar
4,AUT,Austria,Austria,Republic of Austria,Euro


In [9]:
AccountsData = pd.read_csv("Datasets/Wealth-AccountData.csv")

In [10]:
AccountsData.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1995,1996,1997,1998,1999,2000,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,4.49E+10,4.34E+10,3.71E+10,3.88E+10,4.22E+10,4.36E+10,...,6.610000e+10,6.810000e+10,6.850000e+10,7.080000e+10,7.160000e+10,7.250000e+10,7.370000e+10,7.57E+10,7.80E+10,8.12E+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,13708.99,11794.1,12399.48,13573.26,14118.13,...,2.258200e+04,2.336286e+04,2.359058e+04,2.439397e+04,2.473162e+04,2.509002e+04,2.559371e+04,26326.26,27160.86,28333.68
2,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,3468.858,3355.549,2868.997,3013.391,3259.893,3424.007,...,2.825336e+03,2.889150e+03,2.848523e+03,2.930234e+03,2.954273e+03,2.957420e+03,2.987545e+03,3072.504,3178.417,3312.927
3,Albania,ALB,"Human capital per capita, employed male (const...",NW.HCA.MEMP.PC,6507.931,6353.737,5466.692,5901.072,6492.73,6872.019,...,1.525319e+04,1.578992e+04,1.579854e+04,1.647360e+04,1.681013e+04,1.699138e+04,1.730701e+04,17817.1,18445.94,19245.19
4,Albania,ALB,"Human capital per capita, female (constant 201...",NW.HCA.FEMA.PC,4893.006,4737.857,4059.321,4191.445,4536.972,4695.071,...,3.529144e+03,3.613606e+03,3.603694e+03,3.683810e+03,3.696744e+03,3.719608e+03,3.767628e+03,3872.142,3992.188,4161.137


In [11]:
AccountsData.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003',
       '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018'],
      dtype='object')

In [12]:
# AccountsData = AccountsData.drop(['2017'], axis=1)
# axis=1 means it's a column
# axis=0 means it's a row
AccountsData_clean = AccountsData[['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', '1995', '2000', '2005', '2010', '2014']]

In [13]:
AccountsData_clean.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1995,2000,2005,2010,2014
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,4.49E+10,4.36E+10,5.440000e+10,6.810000e+10,7.250000e+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,14118.13,1.807230e+04,2.336286e+04,2.509002e+04
2,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,3468.858,3424.007,2.222115e+03,2.889150e+03,2.957420e+03
3,Albania,ALB,"Human capital per capita, employed male (const...",NW.HCA.MEMP.PC,6507.931,6872.019,1.168935e+04,1.578992e+04,1.699138e+04
4,Albania,ALB,"Human capital per capita, female (constant 201...",NW.HCA.FEMA.PC,4893.006,4695.071,2.886736e+03,3.613606e+03,3.719608e+03


In [14]:
AccountsSeries = pd.read_csv("Datasets/Wealth-AccountSeries.csv")

In [15]:
AccountsSeries.columns

Index(['Series Code', 'Topic', 'Indicator Name', 'Short definition', 'Source',
       'Topic.1', 'Unit of measure', 'Periodicity', 'Reference period',
       'Statistical concept and methodology', 'Previous Indicator Code',
       'Previous Indicator Name', 'Unnamed: 12', 'Short definition.1'],
      dtype='object')

In [16]:
AccountsSeries = AccountsSeries[['Series Code','Topic', 'Indicator Name', 'Short definition']]

In [17]:
AccountsSeries.head()

,Series Code,Topic,Indicator Name,Short definition
0,NW.HCA.TO,Human capital (constant 2018 US$),Human capital is computed as the present value...,NaN
1,NW.HCA.PC,Human capital per capita (constant 2018 US$),Human capital is computed as the present value...,NaN
2,NW.HCA.FEMP.PC,"Human capital per capita, employed female (con...",Human capital is computed as the present value...,NaN
3,NW.HCA.MEMP.PC,"Human capital per capita, employed male (const...",Human capital is computed as the present value...,NaN
4,NW.HCA.FEMA.PC,"Human capital per capita, female (constant 201...",Human capital is computed as the present value...,NaN


In [18]:
cur, conn = create_database()

In [19]:
songplay_table_create = ("""CREATE TABLE IF NOT EXISTS AccountsCountry(
country_code VARCHAR,
short_name VARCHAR,
table_name VARCHAR,
long_name VARCHAR,
currency_unit VARCHAR
)""")

In [20]:
cur.execute(songplay_table_create)
conn.commit()

In [21]:
accounts_data_table_create = ("""CREATE TABLE IF NOT EXISTS accountsdata(
country_name VARCHAR,
country_code VARCHAR,
indicator_name VARCHAR,
indicator_code VARCHAR,
year_1995 varchar,
year_2000 varchar,
year_2005 varchar,
year_2010 varchar,
year_2014 varchar
)""")
cur.execute(accounts_data_table_create)
conn.commit()

In [22]:
accountsseries_data_table_create = ("""CREATE TABLE IF NOT EXISTS accountsseries(
series_code VARCHAR,
topic VARCHAR,
indicator_name VARCHAR,
short_definition VARCHAR
)""")
cur.execute(accountsseries_data_table_create)
conn.commit()

In [23]:
accounts_country_table_insert = ("""INSERT INTO accountscountry(
country_code,
short_name,
table_name,
long_name,
currency_unit)
VALUES (%s, %s, %s, %s, %s)
""")

In [24]:
for i, row in AccountsCountry_clean.iterrows():
    cur.execute(accounts_country_table_insert, list(row))
    # there is an error when primary key is used for country_code
    # however, when i remove primary key, then the error disappears
    # the error was about UniqueViolation

In [25]:
conn.commit()

In [26]:
accounts_data_table_insert = ("""INSERT INTO accountsdata(
country_name,
country_code,
indicator_name,
indicator_code,
year_1995,
year_2000,
year_2005,
year_2010,
year_2014)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
""")

In [27]:
for i, row in AccountsData_clean.iterrows():
        cur.execute(accounts_data_table_insert, list(row))

In [28]:
accountseries_data_table_insert = ("""INSERT INTO accountsseries(
series_code,
topic,
indicator_name,
short_definition)
VALUES (%s, %s, %s, %s)
""")

In [29]:
for i, row in AccountsSeries.iterrows():
    cur.execute(accountseries_data_table_insert, list(row))

In [30]:
conn.commit()